# SYDE 556/750 --- Assignment 3
**Student ID: 20823934**

*Note:* Please include your numerical student ID only, do *not* include your name.

*Note:* Refer to the [PDF](https://github.com/celiasmith/syde556-f22/raw/master/assignments/assignment_03/syde556_assignment_03.pdf) for the full instructions (including some hints), this notebook contains abbreviated instructions only. Cells you need to fill out are marked with a "writing hand" symbol. Of course, you can add new cells in between the instructions, but please leave the instructions intact to facilitate marking.

In [ ]:
# Import numpy and matplotlib -- you shouldn't need any other libraries
import numpy as np
import matplotlib.pyplot as plt

# Fix the numpy random seed for reproducible results
np.random.seed(18945)

# Some formating options
%config InlineBackend.figure_formats = ['svg']

# 1. Decoding from a population

**a) Tuning curves.** Plot the tuning curves (firing rate of each neuron for different $x$ values between $-2$ and $2$).

In [ ]:
""" Set up the variables """

N = 100         # Number of samples per neuron
n = 20          # Number of neurons
r = 2           # Radius
tau_ref = 0.002 # 2 ms
tau_rc = 0.02   # 20 ms
np.random.seed(0)

x = np.linspace(-r, r, N)                               # x-axis
a_max = np.random.uniform(low=100, high=200, size=n)    # Firing rates
xi = np.random.uniform(low=-r, high=r, size=n)          # x-intercepts
e = np.random.choice([-1, 1], size=n)                   # Encoder directions

""" Compute alpha and J^bias """

def G_inverse(a):
    return 1 / (1 - np.exp((tau_ref - 1/a)/(tau_rc)))

def G(J):
    if J > 1:
        return 1 / (tau_ref - tau_rc * np.log(1 - 1/J))
    else:
        return 0
G_vec = np.vectorize(G)

# Slightly change the formula for alpha to account for the radius
alpha = (G_inverse(a_max) - 1) / (r - xi)
J_bias = 1 - alpha * xi

""" Calculate the tuning curves """

A = []
for i in range(n):
    a = G_vec(e[i] * alpha[i] * x + J_bias[i])
    A.append(a)
A = np.array(A)

""" Plotting the curves """

def plot_A(x, A):
    for i in range(A.shape[0]):
        plt.plot(x, A[i])

    plt.title(f"{A.shape[0]} LIF Tuning Curves")
    plt.xlabel("Represented Value x")
    plt.ylabel("Firing Rate (Hz)")
    plt.show()

plot_A(x, A)

**b) Decoder and error computation.** Compute the decoders and plot $(x-\hat{x})$. When computing decoders, take into account noise ($\sigma=0.1 \cdot 200\,\mathrm{Hz}$). When computing $\hat{x}$, add random Gaussian noise with $\sigma=0.1 \cdot 200\,\mathrm{Hz}$ to the activity. Report the Root Mean-Squared Error (RMSE).

In [ ]:
def plot_decoder_error(x, D, A):
    # Compute the decoded value
    x_hat = D @ A

    # Report the RMSE
    rmse = np.sqrt(np.mean((x_hat - x)**2))
    rmse = round(rmse, 3)
    print(f"RMSE: {rmse}")

    # Plot the error
    plt.plot(x, x - x_hat)
    plt.title("Error x - $\\hat{x}$")
    plt.xlabel("Original x")
    plt.ylabel("Decoder Error")
    plt.show()

    return rmse

# Generate the noise
np.random.seed(0)
mu = 0
sigma = 0.1 * 200
noise = np.random.normal(mu, sigma, A.shape)

# Add noise to the activities but don't allow negative firing rates
A_noisy = np.maximum(A + noise, 0)
plot_A(x, A_noisy)

# Find the regularized decoder
D_reg = A @ x @ np.linalg.inv(A @ A.T + N * np.square(sigma) * np.eye(n))

# Plot the decoded signal and report the RMSE
_ = plot_decoder_error(x, D_reg, A_noisy)

# 2. Decoding from two spiking neurons

**a) Synaptic filter.** Plot the post-synaptic current
		$$
			h(t)= \begin{cases}
				0 & \text{if } t < 0 \,, \\
				\frac{e^{-t/\tau}}{\int_0^\infty e^{-t'/\tau} \mathrm{d}t'} & \text{otherwise} \,.
			\end{cases}
		$$

In [ ]:
def post_synaptic_current(tau, a=-0.005, b=0.05, dt=0.001):
    t = np.arange(a, b, dt)
    h = np.exp(-t/tau)

    for i, time in enumerate(t):
        if time < 0:
            h[i] = 0

    h /= np.sum(h)  # Normalize h to have area 1
    return t, h

# h(t) with a time constant of τ = 5 ms
t, h = post_synaptic_current(tau=0.005)

plt.plot(t, h)
plt.xlabel('Time (s)')
plt.ylabel('Magnitude')
plt.title('Synaptic Filter')
plt.show()

**b) Decoding using a synaptic filter.** Plot the original signal $x(t)$, the spikes, and the decoded $\hat{x}(t)$ all on the same graph.

In [ ]:
""" Generate the two opposite neurons """

# Use the same alpha and J_bias for both
neuron_idx = 9
chosen_alpha = alpha[neuron_idx]
chosen_J_bias = J_bias[neuron_idx]
print(f"Neuron {neuron_idx} has a firing rate of {np.round(A[neuron_idx][N//2])} Hz at x = 0")
print(f"alpha = {np.round(chosen_alpha, 3)}, J_bias = {np.round(chosen_J_bias, 3)}")

plot_A(x, np.array([G_vec(chosen_alpha * x + chosen_J_bias), G_vec(-chosen_alpha * x + chosen_J_bias)]))

""" Generate a random input x(t) """

T = 1
dt = 0.001

def generate_signal(T, dt, rms, limit, seed):
    np.random.seed(seed)
    ts = np.arange(0, T, dt)                                # Time points
    fs_hz = np.fft.fftshift(np.fft.fftfreq(len(ts), dt))    # Frequency bins in Hz
    fs_rad = 2 * np.pi * fs_hz                              # Frequency bins in rad/s

    # Generate half of the random signal in the frequency domain
    num_samples = len(fs_rad) // 2
    real = np.random.normal(0, 1, num_samples)
    imag = np.random.normal(0, 1, num_samples)
    half_freq_signal = real + 1j * imag

    # Create the full frequency signal by mirroring the half signal
    # Note: The DC component is 0, so the mean of the time domain signal is 0
    freq_signal = np.concatenate((half_freq_signal, np.array([0]), np.conj(np.flip(half_freq_signal))))

    # Cut freq_signal so it has the same length as fs (in case len(fs) is even)
    freq_signal = freq_signal[:len(fs_rad)]

    # Limit the signal to the desired frequency range
    freq_signal[np.abs(fs_hz) > limit] = 0

    # Turn it back into the time domain
    time_signal = np.fft.ifft(np.fft.ifftshift(freq_signal))

    # Check if the time domain signal is real
    imag_threshold = 1e-10  # Error threshold
    if np.all(np.abs(time_signal.imag) < imag_threshold):
        time_signal = time_signal.real
    else:
        raise ValueError("The time domain signal is not real")

    # Scale the time and frequency signals to the RMS power
    old_rms = np.sqrt(np.mean(time_signal ** 2))
    scaling_factor = rms / old_rms
    time_signal *= scaling_factor
    freq_signal *= scaling_factor

    return ts, fs_rad, time_signal, freq_signal

# 1 s long, dt = 1 ms, with rms = 1, and an upper limit of 5 Hz
ts, fs_rad, time_signal, freq_signal = generate_signal(T=T, dt=dt, rms=1, limit=5, seed=0)

""" Feed the signal into the two neurons and generate spikes """

def spike_train(x, e, alpha, J_bias, tau_ref, tau_rc, v_threshold=1, T=1, dt=0.001):
    ts = np.arange(0, T, dt)
    voltage = 0
    refractory_countdown = 0
    J = alpha * e * x + J_bias

    spikes = []
    voltages = []
    for i in range(len(ts)):
        # Make sure the voltage is non-negative
        voltage = max(voltage, 0)
        voltages.append(voltage)
        did_spike = 0

        # If not in refractory period, update the voltage
        if refractory_countdown <= 0:
            # Update the voltage
            delta_v = (J[i] - voltage) / tau_rc # dv/dt = (J - v) / tau_rc
            voltage += delta_v * dt             # dv = dv/dt * dt

            # If there's a spike, reset the voltage and start the countdown
            if voltage >= v_threshold:
                did_spike = 1
                voltage = 0
                refractory_countdown = tau_ref

        # If in refractory period, only update the countdown
        else:
            refractory_countdown -= dt
        
        spikes.append(did_spike)

    spikes = np.array(spikes)
    num_spikes = sum(spikes)

    return ts, spikes, num_spikes, voltages


# Neurons with opposite encoder directions
ts, spikes_pos, num_spikes_pos, voltages = spike_train(x=time_signal, e=1, alpha=chosen_alpha, J_bias=chosen_J_bias, tau_ref=tau_ref, tau_rc=tau_rc, T=T, dt=dt)
ts, spikes_neg, num_spikes_neg, voltages = spike_train(x=time_signal, e=-1, alpha=chosen_alpha, J_bias=chosen_J_bias, tau_ref=tau_ref, tau_rc=tau_rc, T=T, dt=dt)

""" Decode the spikes back into x_hat(t) """

# Calculate the decoder for average firing rates
def activity_decoder(x, e, alpha, J_bias, sigma):
    A = []
    for i, dir in enumerate(e):
        a = G_vec(dir * alpha[i] * x + J_bias[i])
        A.append(a)
    A = np.array(A)

    D = A @ x @ np.linalg.inv(A @ A.T + len(x) * np.square(sigma) * np.eye(len(e)))
    xhat = D @ A
    rmse = np.sqrt(np.mean((xhat - x)**2))

    return D, rmse, xhat

# Use the decoder from average firing rates to decode the spikes
def decode_spike_trains(ts, x, D, spikes, h, dt):
    # Build the A matrix by filtering the spikes with h(t)
    A = []
    for neuron in spikes:
        a = np.zeros_like(ts)
        for i, time in enumerate(ts):
            if neuron[i] == 1:
                for j in range(100):
                    if i+j < len(a):
                        a[i+j] += h[j]
        A.append(a)
    A = np.array(A)

    # Decode the spikes and scale the output by time step dt
    xhat = D[np.newaxis, :] @ A / dt
    rmse = np.sqrt(np.mean((xhat - x)**2))

    return xhat[0], rmse

t, h = post_synaptic_current(tau=0.005, a=0, b=0.1, dt=dt)
D, _, _ = activity_decoder(x=time_signal, e=[1, -1], alpha=[chosen_alpha]*2, J_bias=[chosen_J_bias]*2, sigma=sigma)
xhat, two_neuron_rmse = decode_spike_trains(ts, x=time_signal, D=D, spikes=[spikes_pos, spikes_neg], h=h, dt=dt)

""" Plot the original signal x(t), the spikes, and the decoded x_hat(t) all on the same graph """

# Plot x(t) and xhat(t)
plt.figure(figsize=(12, 5))
plt.plot(ts, time_signal, label="x(t)")
plt.plot(ts, xhat, label="$\\hat{x}(t)$")

# Plot the spikes as dots
pos_spike_idx = np.where(spikes_pos == 1)[0]
neg_spike_idx = np.where(spikes_neg == 1)[0]
pos_spike_times = ts[pos_spike_idx]
neg_spike_times = ts[neg_spike_idx]
plt.scatter(pos_spike_times, np.ones_like(pos_spike_times), color='blue', marker='.', label='Positive Spikes')
plt.scatter(neg_spike_times, -np.ones_like(neg_spike_times), color='red', marker='.', label='Negative Spikes')

plt.legend()
plt.xlabel('Time (s)')
plt.ylabel('Magnitude')
plt.title('Original and Decoded Signals + Spikes')
plt.show()

**c) Error analysis.** Compute the RMSE of the decoding.

In [ ]:
print(f"RMSE: {two_neuron_rmse}")

# 3. Decoding from many neurons

**a) Exploring the error for an increasing neuron count.** Plot the Root Mean-Squared Error as the number of neurons increases, on a log-log plot. Try $8$ neurons, $16$ neurons, $32$, $64$, $128$, up to $256$. For the RMSE for a particular number of neurons, average over at least $5$ randomly generated groups of neurons. For each group of neurons, randomly generate the signal $x(t)$. Use the same parameters as in question 2.

In [ ]:
def neuron_group_rmse(n, seed):
    # Set up the neurons
    np.random.seed(seed)

    T = 1           # Simulation time
    dt = 0.001      # Step size
    r = 2           # Radius
    tau_ref = 0.002 # 2 ms
    tau_rc = 0.02   # 20 ms
    sigma = 20      # 0.1 * 200

    a_max = np.random.uniform(low=100, high=200, size=n)    # Firing rates
    xi = np.random.uniform(low=-2, high=2, size=n)          # x-intercepts
    e = np.random.choice([-1, 1], size=n)                   # Encoder directions

    alpha = (G_inverse(a_max) - 1) / (r - xi)
    J_bias = 1 - alpha * xi

    # Prepare filter h(t) and random signal x(t)
    t, h = post_synaptic_current(tau=0.005, a=0, b=0.1, dt=dt)
    ts, fs_rad, time_signal, freq_signal = generate_signal(T=T, dt=dt, rms=1, limit=5, seed=seed)

    # Compile the spike trains
    all_spikes = []
    for i in range(n):
        ts, spikes, num_spikes, voltages = spike_train(x=time_signal, e=e[i], alpha=alpha[i], J_bias=J_bias[i], tau_ref=tau_ref, tau_rc=tau_rc, T=T, dt=dt)
        all_spikes.append(spikes)

    D, act_rmse, act_xhat = activity_decoder(x=time_signal, e=e, alpha=alpha, J_bias=J_bias, sigma=sigma)
    spike_xhat, spike_rmse = decode_spike_trains(ts, x=time_signal, D=D, spikes=all_spikes, h=h, dt=dt)
    
    return spike_rmse, spike_xhat, act_rmse, act_xhat, time_signal

def compare_errors(power_low=3, power_high=8, num_trials=10, runs_per_trial=5):
    # Logarithmically space the number of neurons
    ns = np.logspace(power_low, power_high, num_trials, base=2)
    ns = np.round(ns).astype(int)

    # Average the RMSE over 5 runs for each n
    seed = 0
    avg_spike_rmses = []
    avg_act_rmses = []
    for n in ns:
        avg_spike_rmse = 0
        avg_act_rmse = 0
        for i in range(runs_per_trial):
            spike_rmse, spike_xhat, act_rmse, act_xhat, time_signal = neuron_group_rmse(n=n, seed=seed)
            avg_spike_rmse += spike_rmse / runs_per_trial
            avg_act_rmse += act_rmse / runs_per_trial
            seed += 1
        avg_spike_rmses.append(avg_spike_rmse)
        avg_act_rmses.append(avg_act_rmse)

    # Plot the RMSE vs. n
    plt.plot(ns, avg_spike_rmses, label="Spiking Neurons")
    plt.plot(ns, 1/np.sqrt(ns), label=r"$\frac{1}{\sqrt{N}}$")
    plt.plot(ns, avg_act_rmses, label="Activity Neurons")
    plt.plot(ns, 1/ns, label="1/N")
    plt.xscale('log', base=10)
    plt.yscale('log', base=10)
    plt.legend()
    plt.title("RMSE vs. Number of Neurons")
    plt.xlabel("Number of Neurons n")
    plt.ylabel("RMSE")
    plt.show()

    # Plot the final signal and decoded signal for the highest n
    plt.figure(figsize=(12, 5))
    plt.plot(time_signal, label="$x(t)$")
    plt.plot(spike_xhat, label="Spike $\\hat{x}(t)$")
    plt.plot(act_xhat, label="Activity $\\hat{x}(t)$")
    plt.legend()
    plt.xlabel("Time (s)")
    plt.ylabel("Magnitude")
    plt.title(f"Signals for n = {2**power_high} Neurons (Spike RMSE = {np.round(spike_rmse, 3)}, Activity RMSE = {np.round(act_rmse, 3)})")
    plt.show()

compare_errors()

**b) Discussion.** Discuss your results. What is the systematic relationship between the neuron count and the error?

The RMSE goes down as the number of neurons increases. The reason for the decreasing error is shown in the second graph, where $x(t)$ and $\hat{x}(t)$ for 256 neurons are visualized. Compared to the graph in 2b) with only 2 neurons, the decoded spike signal in this graph fits the original signal almost perfectly, despite the filter $h(t)$ having a sharp peak at each spike. Due to the causal nature of the filter, the error can only be completely eliminated using infinite neurons.

Unfortunately, only the activity neurons have an error curve proportional to $\frac{1}{N}$, with the spiking neurons being closer to $\frac{1}{\sqrt{N}}$. However, this slope is not unusual and the linear nature of the relationship is maintained.

# 4. Connecting two groups of neurons

**a) Computing a function.** Show the behaviour of the system with an input of $x(t)=t-1$ for $1\,\mathrm{s}$ (a linear ramp from $-1$ to $0$). Plot the ideal $x(t)$ and $y(t)$ values, along with $\hat{y}(t)$.

In [ ]:
def create_neuron_group(n, r, sigma, h, signal, tau_ref, tau_rc, T, dt, func=None, seed=0):
    np.random.seed(seed)
    N = r * 100                                             # Number of samples per neuron
    x = np.linspace(-r, r, N)                               # x-axis
    a_max = np.random.uniform(low=100, high=200, size=n)    # Firing rates
    xi = np.random.uniform(low=-r, high=r, size=n)          # x-intercepts
    e = np.random.choice([-1, 1], size=n)                   # Encoder directions

    alpha = (G_inverse(a_max) - 1) / (r - xi)
    J_bias = 1 - alpha * xi

    A = []
    for i in range(n):
        a = G_vec(e[i] * alpha[i] * x + J_bias[i])
        A.append(a)
    A = np.array(A)

    # plot_A(x, A)

    # Compute the decoder for f(x)
    if func is not None:
        x = func(x)
    D = A @ x @ np.linalg.inv(A @ A.T + len(x) * np.square(sigma) * np.eye(n))

    # Calculate and decode the spike train
    all_spikes = []
    for i in range(n):
        ts, spikes, _, _ = spike_train(x=signal, e=e[i], alpha=alpha[i], J_bias=J_bias[i], tau_ref=tau_ref, tau_rc=tau_rc, T=T, dt=dt)
        all_spikes.append(spikes)
    spike_xhat, spike_rmse = decode_spike_trains(ts, x=signal, D=D, spikes=all_spikes, h=h, dt=dt)

    return spike_xhat, spike_rmse

def func_4a(T=1, dt=0.001):
    ts = np.arange(0, T, dt)
    signal = ts - 1         # x(t) = t - 1
    answer = 2 * signal + 1 # y(t) = 2x + 1
    return ts, signal, answer

def multiple_neuron_groups(func):
    # Define the variables
    T = 1               # Simulation time
    dt = 0.001          # Step size
    n = 200             # Number of neurons
    r = 1               # Radius
    tau_ref = 0.002     # 2 ms
    tau_rc = 0.02       # 20 ms
    sigma = 0.1 * 200   # Assume a_max = 200

    # Get the ideal signal and answer
    ts, signal, answer = func(T=T, dt=dt)

    t, h = post_synaptic_current(tau=0.005, a=0, b=0.1, dt=dt)
    xhat, _ = create_neuron_group(n=n, r=r, sigma=sigma, h=h, signal=signal, tau_ref=tau_ref, tau_rc=tau_rc, T=T, dt=dt, func=lambda x: 2*x+1, seed=0)
    yhat, _ = create_neuron_group(n=n, r=r, sigma=sigma, h=h, signal=xhat, tau_ref=tau_ref, tau_rc=tau_rc, T=T, dt=dt, func=None, seed=1)

    # Plot the results
    plt.figure(figsize=(10, 5))
    plt.plot(ts, signal, label="$x(t)$")
    plt.plot(ts, answer, label="$y(t)$")
    plt.plot(ts, yhat, label="$\\hat{y}(t)$")
    plt.legend()
    plt.xlabel("Time (s)")
    plt.ylabel("Magnitude")
    plt.title("Ideal and Decoded Signals")
    plt.show()

multiple_neuron_groups(func=func_4a)

**b) Step input.** Repeat part (a) with an input that is ten randomly chosen values between -1 and 0, each one held for 0.1 seconds (a randomly varying step input)

In [ ]:
def func_4b(T=1, dt=0.001, step_size=0.1, seed=4):
    np.random.seed(seed)

    n_steps = int(T / step_size)
    steps = np.random.uniform(-1, 0, size=n_steps)
    ts = np.arange(0, T, dt)

    signal = np.zeros_like(ts)
    for i in range(n_steps):
        signal[int(i*step_size/dt):int((i+1)*step_size/dt)] = steps[i]

    answer = 2 * signal + 1
    return ts, signal, answer

multiple_neuron_groups(func=func_4b)

**c) Sinusoidal input.** Repeat part (a) with an input that is $x(t)=0.2\sin(6\pi t)$.

In [ ]:
def func_4c(T=1, dt=0.001):
    ts = np.arange(0, T, dt)
    signal = 0.2 * np.sin(6 * np.pi * ts)
    answer = 2 * signal + 1
    return ts, signal, answer

multiple_neuron_groups(func=func_4c)

**d) Discussion.** Briefly discuss the results for this question. Does the output match the ideal output? What kind of deviations do you see and why do those exist?

✍ \<YOUR SOLUTION HERE\>

# 5. Connecting three groups of neurons

**a) Sinusoidal input.** Plot $x(t)$, $y(t)$, the ideal $z(t)$, and the decoded $\hat{z}(t)$ for an input of $x(t)=\cos(3\pi t)$ and $y(t)=0.5 \sin (2 \pi t)$ (over $1\,\mathrm{s}$).

In [ ]:
# ✍ <YOUR SOLUTION HERE>

**b) Random input.** Plot $x(t)$, $y(t)$, the ideal $z(t)$, and the decoded $\hat{z}(t)$ for a random input over $1\,\mathrm{s}$. For $x(t)$ use a random signal with a limit of $8\,\mathrm{Hz}$ and $\mathtt{rms}=1$. For $y(t)$ use a random signal with a limit of $5\,\mathrm{Hz}$ and $\mathtt{rms}=0.5$.

In [ ]:
# ✍ <YOUR SOLUTION HERE>

# 6. Computing with vectors

**a) Constant inputs.** Plot the decoded output $\hat{w}(t)$ and the ideal $w$ for
		$$x =(0.5,1), \quad y = (0.1,0.3), \quad z =(0.2,0.1), \quad q = (0.4,-0.2) \,.$$

In [ ]:
# ✍ <YOUR SOLUTION HERE>

**b) Sinusoidal input.** Produce the same plot for
$$x =(0.5,1), \quad y = (\sin(4\pi t),0.3), \quad z =(0.2,0.1), \quad q = (\sin(4\pi t),-0.2) \,.$$

In [ ]:
# ✍ <YOUR SOLUTION HERE>

**c) Discussion.** Describe your results and discuss why and how they stray from the expected answer.

✍ \<YOUR SOLUTION HERE\>